In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from stop_words import get_stop_words
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV, KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from yellowbrick.features.importances import FeatureImportances
import pickle
import os

### clean

In [2]:
df = pd.read_json('../../../data/uniqueScaledDataset.json')
print(df.shape)
df.head()

(38459, 14)


,viewCount,description,tags,predClass,channelId,videoId,comments,categoryId,commentCount,channelTitle,redirectedDomains,domains,title,completeDomains
0,298744,After months of searching for a constant onlin...,"[Money (TV Episode), How-to (Media Genre), Bus...",f,UCFqanUMpCMh54RmqRFh8hZw,dFotOUq0Z4E,"[over $34,000 was added to my paypal and that ...",22,240,ProfitClicksDelux,"[http://www.neobux.com/?r=dirven, https://imgu...","[neobux, imgur]",How to make money online just by clicking ads ...,"[http://www.neobux.com/?r=dirven, http://imgur..."
1,44244,hii.\nfriends.\n[[[[[[[[[[[[[[[[[[[[[[[[[[[[th...,"[earn money, how to, ads, paer, click, $5, per...",f,UC775w8nPdhDCVJ4IkCczVPw,xO5B6JcR5TQ,"[Earn Money\r\nhttps://youtu.be/NsqaXAMhl3k, n...",22,56,Trick Technology,[],[],earn money $2.to$5. ads click per day. in hindi.,[]
2,1066128,Create Free Bitcoin Wallet:- zebpay:- http://l...,[work from home],f,UCdWrX9xcYhCW4RwwYT22bpg,adh4xyzygBw,"[Sir ye site nhi chal rhi bhad ho gyi h kya, M...",26,2105,helpfromankit,"[http://link.zebpay.com/ref/REFANKI3843, http:...","[link.zebpay, myadvertisingpays]",view Only 10 Ads Per Day and Earn Money Every ...,"[http://link.zebpay.com/ref/REFANKI3843, http:..."
3,7567,My #1 Recommendation To Earning Income Online ...,"[make money online, tyler pratt, tylerpratt, e...",f,UCRfWuSls0R7jQazuddXiPvQ,H1KmFXp2Ymg,"[Didn't work, The real big money?? LOL cl...",26,9,Tyler Pratt,"[https://go.tylerpratt.com/, https://go.tylerp...","[go.tylerpratt, go.tylerpratt, facebook, insta...",Easy Side Online Job Clicking Ads and Making M...,"[http://go.tylerpratt.com/, http://go.tylerpra..."
4,9107,"Please watch: ""how to earn money from internet...","[Earn money By Click Ads, make money online, m...",f,UCupgYB_GTCWX6BLh-Ui_VFg,-YtKEcbVsKE,"[madarchod Frank hi, If you have website and e...",28,19,Tech 4 Digital,"[https://www.youtube.com/watch?v=DBZOdbHELXU, ...","[youtube, youtube, youtube, youtube, youtube, ...",Make money by Click ads क्लिक करके पैसे कमाओ ...,"[https://www.youtube.com/watch?v=DBZOdbHELXU, ..."


In [3]:
df['videoId'].is_unique

True

In [4]:
len(df['channelId'].unique())

22777

In [5]:
df.dtypes

viewCount            object
description          object
tags                 object
predClass            object
channelId            object
videoId              object
comments             object
categoryId           object
commentCount         object
channelTitle         object
redirectedDomains    object
domains              object
title                object
completeDomains      object
dtype: object

In [9]:
# df['classification'] = df['classification'].astype('category')
df['commentCount'][df['commentCount'] == 'No commentCount'] = np.NaN
df['commentCount'] = df['commentCount'].astype('float')
# df['category'] = df['category'].astype('category')
df['viewCount'][df['viewCount'] == 'No viewCount'] = np.NaN
df['viewCount'] = df['viewCount'].astype('float')
# df['linksCount'] = df['linksCount'].astype('float')

C:\Users\ruote\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ruote\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [146]:
df = df[['category', 'channelId', 'channelTitle', 'classification',
       'commentCount', 'comments', 'commonMal', 'completeDomains',
       'description', 'domains', 
       'linkContent', 'links', 'linksCount', 'linksDown', 'linksDownCount',
       'linksUp', 'redirectedDomains', 'tags', 'title',
       'videoId', 'viewCount']]

In [147]:
df.to_pickle('../Data Files/merged.pkl')